# Fine Tuning

#### Como ajustar um modelo que já aprendeu muitas coisas, para que ele possa aprender algo novo mais facilmente
#### Passamos um arquivo com varios exemplos de modelo para que o Assistant responda como nos exemplos.

### Resposta que gostaríamos do modelo

```
{
  "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "AsimoBot"
}
```

### Resposta que o modelo está nos dando

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a é diferente de zero. A incógnita da equação é x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas.

### Prompt para que ele nos dê a mensagem formatada como queremos

system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

### Formatação das mensagens que precisamos para realizarmos o Fine Tuning

```
{"messages": 
    [
        {"role": "user", "content": "O que é uma equação quadrática"},
        {"role": "assistant", "content": 
            {
                "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
                "categoria": "Matemática",
                "fonte": "AsimoBot"
            }},
    ]
}

```

### Abrindo e visualizando o arquivo JSON contendo os exemplos de perguntas e respostas

O arquivo é atribuido a 'f' e o conteudo de 'f' é atribuido, em formato python, a json_respostas

In [1]:
import json

with open('chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)

In [2]:
json_respostas

[{'pergunta': 'Qual é a fórmula da área de um círculo?',
  'resposta': 'A área de um círculo é π vezes o raio ao quadrado (A = πr²).',
  'categoria': 'Matemática'},
 {'pergunta': 'O que é um substantivo?',
  'resposta': 'Substantivo é a classe gramatical de palavras que nomeiam seres, objetos, fenômenos, lugares, qualidades e ações.',
  'categoria': 'Língua Portuguesa'},
 {'pergunta': 'O que é inércia?',
  'resposta': 'Inércia é a tendência de um corpo manter seu estado de repouso ou movimento uniforme, a menos que forças atuem sobre ele.',
  'categoria': 'Física'},
 {'pergunta': 'O que é uma equação do segundo grau?',
  'resposta': 'É uma equação polinomial onde o maior expoente da incógnita é 2, geralmente na forma ax² + bx + c = 0.',
  'categoria': 'Matemática'},
 {'pergunta': 'O que é fotossíntese?',
  'resposta': 'Fotossíntese é o processo pelo qual plantas e outros organismos convertem luz solar em energia química para alimentar suas atividades.',
  'categoria': 'Biologia'},
 {'p

# Precisamos passar esse conteudo para o formato aceito para Fine Tuning

#### Escreve os dados de `json_respostas` em um arquivo JSONL, onde cada linha contém uma entrada  em um formato específico.

In [3]:
# Cria o arquivo Jsonl em modo de escrita ('w') com UTF-8 e atribui o objeto de arquivo resultante à variável f
with open('chatbot_respostas.jsonl', 'w', encoding="utf8") as f:  
    for entrada in json_respostas:  # Itera em cada linha e cria um dicionário (resposta) com as chaves 'resposta', 'categoria' e 'fonte'. 
        resposta = {                # Os valores correspondentes são atribuidos.
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'AsimoBot'} 
        
        
        # Cria um dicionário (entrada_jsonl), com lista de mensagens (um dicionário com um role e content. 
        # O papel pode ser 'user' ou 'assistant'. O conteúdo é a resposta convertida em JSON usando json.dumps().
        
        entrada_jsonl = {  # Mapeia as respostas em formato de mensagem
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        
        # Escreve a entrada JSONL atual (entrada_jsonl) no arquivo f usando json.dump(). 
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        
        f.write('\n')   # Escreve uma nova linha no arquivo f. Separa cada entrada JSONL em linhas distintas.

### Instaniando o Client

In [4]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Passando o JsonL para o OPENAI


'rb': modo de abertura em modo binario. É comumente usado quando está lidando com arquivos que não são texto simples, como imagens, áudio ou outros tipos.

purpose='fine-tune' é um parâmetro que especifica o propósito para o qual o arquivo será usado. Neste caso, indica que o arquivo será usado para "fine-tuning" , ou seja, será utilizado para adaptar o modelo pré-treinado para uma tarefa específica.

In [8]:
# Cria um arquivo usando o conteúdo do `chatbot_respostas.jsonl`, para ser usado para fine-tuning de um modelo.
file = client.files.create(
    file=open('chatbot_respostas.jsonl',  'rb'),
    purpose='fine-tune'
)

#  cria tarefa que usa o arquivo para efetivamente realizar o ajuste fino. inicia o processo de treinamento do modelo
client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-8LvV5M7m8fhkMJ99OlzsZsKH', created_at=1716062338, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-rmzTdLUJPkFyj2Pw8pHNBbGI', result_files=[], seed=1121202986, status='validating_files', trained_tokens=None, training_file='file-l4mLbXTsDv2hWVycH2e0PEcG', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

### Solicitando uma lista de todos os fine-tuning que foram criados no cliente.

In [5]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-8LvV5M7m8fhkMJ99OlzsZsKH', created_at=1716062338, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9QKgWokQ', finished_at=1716063066, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-rmzTdLUJPkFyj2Pw8pHNBbGI', result_files=['file-5vVIXDyeErWEeNeZH0kUaiZP'], seed=1121202986, status='succeeded', trained_tokens=29211, training_file='file-l4mLbXTsDv2hWVycH2e0PEcG', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

# Utilizando o Modelo

### Resposta padrão do modelo

In [6]:
mensagens = [{'role': 'user', 'content': 'O que é uma equação quadrática?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a ≠ 0. A incógnita da equação é x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas.


### Resposta do modelo com prompt melhor

In [7]:
system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

# Acrescentando o prompt
mensagens = [
    {'role': 'system', 'content': system_mes},
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
    "fonte": "AsimoBot",
    "resposta": "Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, com a forma ax^2 + bx + c = 0.",
    "categoria": "matemática"
}


#### A resposta foi como queriamos. Porem, temos um alto Gasto se toda vez formos usar esses prompts para especificar resposta:

In [8]:
resposta.usage

CompletionUsage(completion_tokens=62, prompt_tokens=114, total_tokens=176)

# Resposta do modelo utilizando o Fine Tuning

In [11]:
mensagens = [
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="ft:gpt-3.5-turbo-0125:personal::9QKgWokQ",  # Passar o id do Tuning Modelo 
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
  "resposta": "Uma equação quadrática é uma equação polinomial de segundo grau, geralmente na forma ax^2 + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "AsimoBot"
}


### Podemos notar bem menos tokens utilizados para a resposta como queremos

In [12]:
resposta.usage

CompletionUsage(completion_tokens=60, prompt_tokens=16, total_tokens=76)